<a href="https://colab.research.google.com/github/sesmael/Real-Time-ML-/blob/main/Homework5_problem1_and_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# ---------------------------
# Full English-to-French Dataset (for reference)
# ---------------------------
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur"),
    ("They ride bicycles", "Ils font du vélo"),
    ("The coffee is hot", "Le café est chaud"),
    ("She wears glasses", "Elle porte des lunettes"),
    ("We visit our grandparents", "Nous rendons visite à nos grands-parents"),
    ("He plays the guitar", "Il joue de la guitare"),
    ("They go shopping", "Ils font du shopping"),
    ("The teacher explains the lesson", "Le professeur explique la leçon"),
    ("She takes the train to work", "Elle prend le train pour aller au travail"),
    ("We bake cookies", "Nous faisons des biscuits"),
    ("He washes his hands", "Il se lave les mains"),
    ("They enjoy the sunset", "Ils apprécient le coucher du soleil"),
    ("The river flows calmly", "La rivière coule calmement"),
    ("She feeds the cat", "Elle nourrit le chat"),
    ("We visit the museum", "Nous visitons le musée"),
    ("He fixes his bicycle", "Il répare son vélo"),
    ("They paint the walls", "Ils peignent les murs"),
    ("The baby sleeps peacefully", "Le bébé dort paisiblement"),
    ("She ties her shoelaces", "Elle attache ses lacets"),
    ("We climb the stairs", "Nous montons les escaliers"),
    ("He shaves in the morning", "Il se rase le matin"),
    ("They set the table", "Ils mettent la table"),
    ("The airplane takes off", "L'avion décolle"),
    ("She waters the plants", "Elle arrose les plantes"),
    ("We practice yoga", "Nous pratiquons le yoga"),
    ("He turns off the light", "Il éteint la lumière"),
    ("They play video games", "Ils jouent aux jeux vidéo"),
    ("The soup smells delicious", "La soupe sent délicieusement bon"),
    ("She locks the door", "Elle ferme la porte à clé"),
    ("We enjoy a picnic", "Nous profitons d'un pique-nique"),
    ("He checks his email", "Il vérifie ses emails"),
    ("They go to the gym", "Ils vont à la salle de sport"),
    ("The moon shines brightly", "La lune brille intensément"),
    ("She catches the bus", "Elle attrape le bus"),
    ("We greet our neighbors", "Nous saluons nos voisins"),
    ("He combs his hair", "Il se peigne les cheveux"),
    ("They wave goodbye", "Ils font un signe d'adieu")
]

# ---------------------------
# 1. Preparing the Next Character Prediction Dataset
# ---------------------------
sequence_text = (
    "Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting "
    "the next character in a sequence of text based on the characters that precede it. This task is essential for various "
    "applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable "
    "of generating human-like text. At its core, next character prediction relies on statistical models or deep learning algorithms "
    "to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns "
    "and relationships learned from large datasets of text during the training phase of the model. One of the most popular approaches to "
    "next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long "
    "Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain "
    "information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability "
    "by being able to remember long-term dependencies, making them even more effective for next character prediction tasks. "
    "Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability "
    "of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters "
    "to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. "
    "Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters "
    "that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion "
    "features, and enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction plays "
    "a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. "
    "Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities "
    "for the future of text-based technology."
)

# Build a character vocabulary and map characters to indices
vocab = sorted(set(sequence_text))
vocab_size = len(vocab)
char_to_idx = {ch: i for i, ch in enumerate(vocab)}
idx_to_char = {i: ch for i, ch in enumerate(vocab)}

def text_to_indices(text):
    return [char_to_idx[ch] for ch in text]

data_indices = text_to_indices(sequence_text)

def create_dataset(seq_len):
    X, y = [], []
    for i in range(len(data_indices) - seq_len):
        X.append(data_indices[i:i+seq_len])
        y.append(data_indices[i+seq_len])
    return np.array(X), np.array(y)

class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------------------
# 2. Define the Models
# ---------------------------
# Transformer-based model for next character prediction
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=4, hidden_dim=256, num_layers=2, seq_len=20, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embed_dim = embed_dim
        self.seq_len = seq_len
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Parameter(torch.zeros(1, seq_len, embed_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)           # (batch, seq_len, embed_dim)
        x = x + self.pos_embedding       # add positional info
        x = x.transpose(0, 1)            # (seq_len, batch, embed_dim)
        x = self.transformer_encoder(x)
        out = x[-1]                     # last time step
        out = self.fc_out(out)
        return out

# Standard LSTM-based model for next character prediction
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.lstm(x)
        out = self.fc_out(output[:, -1, :])
        return out

# LSTM with Attention (without a separate decoder) for next character prediction
class LSTMWithAttention(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, dropout=0.0):
        super(LSTMWithAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=4, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)                    # (batch, seq_len, hidden_dim)
        lstm_out = lstm_out.transpose(0, 1)             # (seq_len, batch, hidden_dim)
        query = lstm_out[-1].unsqueeze(0)              # (1, batch, hidden_dim)
        attn_output, _ = self.attention(query, lstm_out, lstm_out)
        attn_output = attn_output.squeeze(0)           # (batch, hidden_dim)
        out = self.fc_out(attn_output)
        return out

# Encoder-Decoder with Cross-Attention
class EncoderDecoderWithAttention(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, dropout=0.0):
        super(EncoderDecoderWithAttention, self).__init__()
        # Shared embedding for both encoder and decoder
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Encoder: processes the input sequence.
        self.encoder = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)

        # Decoder: a single-step LSTM decoder.
        self.decoder = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        # Learned start token for the decoder input.
        self.start_token = nn.Parameter(torch.randn(1, embed_dim))

        # Cross-attention: decoder queries attend to encoder outputs.
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=4, dropout=dropout)

        # Final prediction layer.
        self.fc_out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        # x: (batch, seq_len)
        batch_size = x.size(0)

        # Encode input sequence.
        encoder_emb = self.embedding(x)                   # (batch, seq_len, embed_dim)
        encoder_outputs, (hn, cn) = self.encoder(encoder_emb)  # (batch, seq_len, hidden_dim)

        # Prepare decoder input: use learned start token for each batch.
        decoder_input = self.start_token.expand(batch_size, 1, -1)  # (batch, 1, embed_dim)
        decoder_output, _ = self.decoder(decoder_input, (hn, cn))    # (batch, 1, hidden_dim)

        # Prepare for attention: query from decoder, keys and values from encoder outputs.
        decoder_output = decoder_output.transpose(0, 1)       # (1, batch, hidden_dim)
        encoder_outputs_t = encoder_outputs.transpose(0, 1)   # (seq_len, batch, hidden_dim)

        attn_output, _ = self.attention(decoder_output, encoder_outputs_t, encoder_outputs_t)
        attn_output = attn_output.squeeze(0)                  # (batch, hidden_dim)
        output = self.fc_out(attn_output)                     # (batch, vocab_size)
        return output

# ---------------------------
# 3. Training and Evaluation Functions
# ---------------------------
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in dataloader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == batch_y).sum().item()
            total += batch_y.size(0)
    return correct / total

# ---------------------------
# 4. Experiment Setup and Training Loop
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20
batch_size = 64
learning_rate = 1e-3
seq_lengths = [10, 20, 30]

final_results = {}

for seq_len in seq_lengths:
    print(f"\n--- Sequence Length: {seq_len} ---")
    X, y = create_dataset(seq_len)
    split_idx = int(0.8 * len(X))
    X_train, y_train = X[:split_idx], y[:split_idx]
    X_val, y_val = X[split_idx:], y[split_idx:]

    train_dataset = TextDataset(X_train, y_train)
    val_dataset = TextDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    models = {
        "Transformer": TransformerModel(vocab_size, seq_len=seq_len).to(device),
        "LSTM": LSTMModel(vocab_size).to(device),
        "LSTM+Attn": LSTMWithAttention(vocab_size).to(device),
        "EncDec+Attn": EncoderDecoderWithAttention(vocab_size).to(device)
    }

    final_results[seq_len] = {}

    for model_name, model in models.items():
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        print(f"\nTraining {model_name} for sequence length {seq_len}:")
        start_time = time.time()
        final_train_loss = None

        for epoch in range(num_epochs):
            train_loss = train_model(model, train_loader, criterion, optimizer, device)
            val_acc = evaluate_model(model, val_loader, device)
            final_train_loss = train_loss
            if (epoch + 1) % 5 == 0 or epoch == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")

        total_time = time.time() - start_time
        final_val_acc = evaluate_model(model, val_loader, device)

        final_results[seq_len][model_name] = {
            "Final_Train_Loss": final_train_loss,
            "Final_Val_Accuracy": final_val_acc,
            "Total_Time_sec": total_time
        }
        print(f"Final {model_name} - Train Loss: {final_train_loss:.4f}, Val Acc: {final_val_acc:.4f}, Time: {total_time:.2f} sec")

print("\n\n--- Overall Comparison of Models ---")
for seq_len, model_results in final_results.items():
    print(f"\nSequence Length: {seq_len}")
    for model_name, metrics in model_results.items():
        print(f"{model_name}: Train Loss = {metrics['Final_Train_Loss']:.4f}, Val Acc = {metrics['Final_Val_Accuracy']:.4f}, Time = {metrics['Total_Time_sec']:.2f} sec")



--- Sequence Length: 10 ---

Training Transformer for sequence length 10:
Epoch 1/20 | Train Loss: 2.9575 | Val Acc: 0.2353
Epoch 5/20 | Train Loss: 1.9924 | Val Acc: 0.2983
Epoch 10/20 | Train Loss: 1.3690 | Val Acc: 0.3361
Epoch 15/20 | Train Loss: 0.9028 | Val Acc: 0.3845
Epoch 20/20 | Train Loss: 0.6365 | Val Acc: 0.3971
Final Transformer - Train Loss: 0.6365, Val Acc: 0.3971, Time: 3.77 sec

Training LSTM for sequence length 10:
Epoch 1/20 | Train Loss: 3.1280 | Val Acc: 0.2143
Epoch 5/20 | Train Loss: 1.8424 | Val Acc: 0.4307
Epoch 10/20 | Train Loss: 1.1317 | Val Acc: 0.4559
Epoch 15/20 | Train Loss: 0.6082 | Val Acc: 0.4643
Epoch 20/20 | Train Loss: 0.2738 | Val Acc: 0.4265
Final LSTM - Train Loss: 0.2738, Val Acc: 0.4265, Time: 1.93 sec

Training LSTM+Attn for sequence length 10:
Epoch 1/20 | Train Loss: 3.1999 | Val Acc: 0.1345
Epoch 5/20 | Train Loss: 2.2872 | Val Acc: 0.3067
Epoch 10/20 | Train Loss: 1.1818 | Val Acc: 0.3908
Epoch 15/20 | Train Loss: 0.4208 | Val Acc: 0.37

In [ ]:
import time
import requests
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# ---------------------------
# Download and Process the Tiny Shakespeare Dataset
# ---------------------------
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # The entire text data

def process_data(sequence_length):
    # Create a character mapping to integers
    chars = sorted(list(set(text)))
    char_to_int = {ch: i for i, ch in enumerate(chars)}
    int_to_char = {i: ch for i, ch in enumerate(chars)}

    # Encode the text into integers
    encoded_text = [char_to_int[ch] for ch in text]

    # Create sequences and targets
    sequences = []
    targets = []
    for i in range(len(encoded_text) - sequence_length):
        seq = encoded_text[i:i + sequence_length]
        target = encoded_text[i + sequence_length]
        sequences.append(seq)
        targets.append(target)
    # Convert lists to PyTorch tensors
    return torch.tensor(sequences, dtype=torch.long), torch.tensor(targets, dtype=torch.long), char_to_int, int_to_char

# ---------------------------
# Define the Dataset Class
# ---------------------------
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# ---------------------------
# Define the Transformer Model for Next Character Prediction
# ---------------------------
class TransformerCharModel(nn.Module):
    def __init__(self, vocab_size, seq_len, embed_dim=128, num_layers=2, num_heads=2, hidden_dim=256, dropout=0.1):
        """
        vocab_size: Number of unique characters
        seq_len: Input sequence length
        embed_dim: Dimension of character embeddings
        num_layers: Number of transformer encoder layers
        num_heads: Number of attention heads
        hidden_dim: Hidden dimension of the feed-forward network
        dropout: Dropout probability
        """
        super(TransformerCharModel, self).__init__()
        self.seq_len = seq_len
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # Learned positional embeddings
        self.pos_embedding = nn.Parameter(torch.zeros(1, seq_len, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim,
                                                   nhead=num_heads,
                                                   dim_feedforward=hidden_dim,
                                                   dropout=dropout,
                                                   batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        # x: (batch_size, seq_len)
        x = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        x = x + self.pos_embedding  # add positional embeddings
        x = self.transformer_encoder(x)  # (batch_size, seq_len, embed_dim)
        # Use the output of the last time step for prediction
        out = self.fc_out(x[:, -1, :])
        return out

# ---------------------------
# Define an RNN Baseline (LSTM) for Comparison
# ---------------------------
class LSTMCharModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, dropout=0.0):
        super(LSTMCharModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc_out(out[:, -1, :])
        return out

# ---------------------------
# Training and Evaluation Functions
# ---------------------------
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in dataloader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == batch_y).sum().item()
            total += batch_y.size(0)
    return correct / total

# ---------------------------
# Experiment Loop
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameter grid for transformers
layer_options = [1, 2, 4]
head_options = [2, 4]
# You might also want to adjust hidden_dim (feed-forward network size) and embed_dim.

sequence_lengths = [20, 30]  # later we will try 50 as well
num_epochs = 10  # set to desired number for experiments
batch_size = 64
learning_rate = 1e-3

# Dictionary to store experiment results
experiment_results = {}

for seq_len in sequence_lengths:
    print(f"\n=== Sequence Length: {seq_len} ===")
    # Process data for current sequence length
    sequences, targets, char_to_int, int_to_char = process_data(seq_len)
    dataset = CharDataset(sequences, targets)
    # Simple train-validation split (80/20)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    vocab_size = len(char_to_int)
    experiment_results[seq_len] = {}

    # Loop over transformer hyperparameters combinations
    for num_layers in layer_options:
        for num_heads in head_options:
            config_name = f"Transformer_layers{num_layers}_heads{num_heads}"
            print(f"\n--- {config_name} ---")
            model = TransformerCharModel(vocab_size, seq_len=seq_len,
                                         embed_dim=128,
                                         num_layers=num_layers,
                                         num_heads=num_heads,
                                         hidden_dim=256,
                                         dropout=0.1).to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            start_time = time.time()
            for epoch in range(num_epochs):
                train_loss = train_model(model, train_loader, criterion, optimizer, device)
                val_acc = evaluate_model(model, val_loader, device)
                print(f"Epoch {epoch+1}/{num_epochs}: Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}")
            elapsed_time = time.time() - start_time
            final_val_acc = evaluate_model(model, val_loader, device)

            # Save results for this configuration
            experiment_results[seq_len][config_name] = {
                "Final_Train_Loss": train_loss,
                "Final_Val_Accuracy": final_val_acc,
                "Training_Time_sec": elapsed_time,
                "Model_Size": sum(p.numel() for p in model.parameters())
            }

    # Train and evaluate the RNN baseline (for comparison)
    print("\n--- RNN Baseline (LSTM) ---")
    rnn_model = LSTMCharModel(vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, dropout=0.0).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)
    start_time = time.time()
    for epoch in range(num_epochs):
        train_loss = train_model(rnn_model, train_loader, criterion, optimizer, device)
        val_acc = evaluate_model(rnn_model, val_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}")
    elapsed_time = time.time() - start_time
    final_val_acc = evaluate_model(rnn_model, val_loader, device)

    experiment_results[seq_len]["RNN_Baseline"] = {
        "Final_Train_Loss": train_loss,
        "Final_Val_Accuracy": final_val_acc,
        "Training_Time_sec": elapsed_time,
        "Model_Size": sum(p.numel() for p in rnn_model.parameters())
    }

# ---------------------------
# Increase Sequence Length to 50 and Report Results
# ---------------------------
seq_len = 50
print(f"\n=== Sequence Length: {seq_len} (Increased) ===")
sequences, targets, char_to_int, int_to_char = process_data(seq_len)
dataset = CharDataset(sequences, targets)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
vocab_size = len(char_to_int)

# Use one configuration (say, 2 layers and 2 heads) for illustration
print("\n--- Transformer (2 layers, 2 heads) with sequence length 50 ---")
model = TransformerCharModel(vocab_size, seq_len=seq_len,
                             embed_dim=128,
                             num_layers=2,
                             num_heads=2,
                             hidden_dim=256,
                             dropout=0.1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
start_time = time.time()
for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, criterion, optimizer, device)
    val_acc = evaluate_model(model, val_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}")
elapsed_time = time.time() - start_time
final_val_acc = evaluate_model(model, val_loader, device)
experiment_results[seq_len] = {
    "Transformer_2layers_2heads": {
        "Final_Train_Loss": train_loss,
        "Final_Val_Accuracy": final_val_acc,
        "Training_Time_sec": elapsed_time,
        "Model_Size": sum(p.numel() for p in model.parameters())
    }
}

print("\n\n--- Experiment Results ---")
for seq_len, results in experiment_results.items():
    print(f"\nSequence Length: {seq_len}")
    for config, metrics in results.items():
        print(f"{config}: Train Loss = {metrics['Final_Train_Loss']:.4f}, Val Acc = {metrics['Final_Val_Accuracy']:.4f}, "
              f"Time = {metrics['Training_Time_sec']:.2f} sec, Model Size = {metrics['Model_Size']} parameters")



=== Sequence Length: 20 ===

--- Transformer_layers1_heads2 ---
Epoch 1/10: Train Loss=2.0604, Val Acc=0.4451
Epoch 2/10: Train Loss=1.9102, Val Acc=0.4558
Epoch 3/10: Train Loss=1.8828, Val Acc=0.4587
Epoch 4/10: Train Loss=1.8711, Val Acc=0.4621
Epoch 5/10: Train Loss=1.8628, Val Acc=0.4669
Epoch 6/10: Train Loss=1.8585, Val Acc=0.4652
Epoch 7/10: Train Loss=1.8561, Val Acc=0.4643
Epoch 8/10: Train Loss=1.8503, Val Acc=0.4652
Epoch 9/10: Train Loss=1.8492, Val Acc=0.4701
Epoch 10/10: Train Loss=1.8478, Val Acc=0.4681

--- Transformer_layers1_heads4 ---
Epoch 1/10: Train Loss=2.0120, Val Acc=0.4656
Epoch 2/10: Train Loss=1.8397, Val Acc=0.4760
Epoch 3/10: Train Loss=1.8083, Val Acc=0.4805
Epoch 4/10: Train Loss=1.7929, Val Acc=0.4816
Epoch 5/10: Train Loss=1.7856, Val Acc=0.4887
Epoch 6/10: Train Loss=1.7797, Val Acc=0.4851
Epoch 7/10: Train Loss=1.7765, Val Acc=0.4893
Epoch 8/10: Train Loss=1.7739, Val Acc=0.4906
Epoch 9/10: Train Loss=1.7694, Val Acc=0.4903
Epoch 10/10: Train Loss=